In [2]:
import sys
import os
sys.path.append('/g/data/jk72/ll6859/ACCESS_aero_eval/')
from aercode import *
import matplotlib 
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import xarray as xr
import glob as gb
import cartopy.crs as ccrs
import seaborn as sns
import scipy.special
from datetime import datetime

import dask
import warnings
from dask.distributed import Client

In [3]:
plt.style.use('default')
warnings.filterwarnings('ignore')

# RVInvestigator

bx400

In [10]:
# load in all observations and resample
f_path = '/g/data/jk72/ll6859/ACCESS_aero_eval/RVInvestigator/'
aer = pd.read_csv(f_path+'rvigaw_cn10_2016to2019_L2.csv', index_col='datetime')
aer['time'] = pd.DatetimeIndex(aer.index)
aer.index = aer['time']
aer = aer.resample('1D', kind='time').mean()
aer = aer[['cn10_avg','cn10_std','latitude','longitude']].dropna()
aer.rename(columns={'latitude': 'lat', 'longitude': 'lon'}, inplace=True)


met = pd.read_csv(f_path+'RVI-met.csv', delimiter=',', skiprows=54, index_col='TIME')
met['time'] = pd.DatetimeIndex(met.index)
met.index = met['time'].astype('datetime64[ns]')
met = met[['AIRT_1','ATMP','RAIN_AMOUNT_1','WDIR_1','WSPD_1']]
met['RAIN_AMOUNT_1'] = met['RAIN_AMOUNT_1'].diff()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']<0]=0
met_tmp = met
met = met.resample('1D', kind='time').mean()
met['RAIN_AMOUNT_1'] = met_tmp['RAIN_AMOUNT_1'].resample('1D', kind='time').sum()
met['RAIN_AMOUNT_1'][met['RAIN_AMOUNT_1']>60]=np.nan
met.rename(columns={'AIRT_1': 'air_temp', 'RAIN_AMOUNT_1': 'total_rain', 'ATMP': 'air_pressure', 'WDIR_1': 'wind_dir', 'WSPD_1': 'wind_spd'}, inplace=True)

rdn = pd.read_excel(f_path+'2020_05_29d_RVI_1420_Radon.xlsx', sheet_name='RVI1420', index_col='DateTime')
rdn['time'] = pd.DatetimeIndex(rdn.index)
rdn.index = rdn['time']
rdn = rdn[rdn['Radon']>0]
rdn = rdn.resample('1D', kind='time').mean()
rdn = rdn[['Radon','[SD]','RnSTP']].dropna()
rdn.rename(columns={'Radon': 'radon', '[SD]': 'radon_std', 'RnSTP': 'radon_stp'}, inplace=True)

In [11]:
# combine all observations
obs = aer.to_xarray()
obs = obs.assign(met)
obs = obs.assign(rdn)

In [14]:
# load in model data
rvi_bx400 = xr.open_dataset(f_path+'rvi1619_bx400_track.nc')

KeyError: 'cn10_avg'

In [ ]:
# merge model and obs
rvi_df = xr.merge([rvi_bx400,obs],compat='override')

# Edit data attributes
rvi_df.attrs['title'] = 'ACCESS-AM2 model output (bx400) and observations along the 2016-2019 RV Investigator voyage tracks'
rvi_df.attrs['contacts'] = 'Liam.Lamprey@utas.edu.au'

In [5]:
rvi_df.load().to_netcdf(path=f_path+'rvi1619_df.nc')

cg283